In [53]:
import pandas as pd
from  geopy.geocoders import Nominatim
import requests
import json
import qwikidata
import qwikidata.sparql

In [183]:
# create dataframe from the Dutch places file

cities = pd.read_csv("WoonplaatsenNL.csv", sep=";")
cities.head()


,ID,Woonplaatsen,Woonplaatscode_1,Woonplaats,Code_3,Provincie,Code_5,Deel,Code_7
0,0,WP1925,WP1925,Almelo,GM0141,Overijssel,PV23,Oost-Nederland,LD02
1,1,WP2774,WP2774,Veere,GM0717,Zeeland,PV29,West-Nederland,LD03
2,2,WP2145,WP2145,Coevorden,GM0109,Drenthe,PV22,Noord-Nederland,LD01
3,3,WP2571,WP2571,Aalsmeer,GM0358,Noord-Holland,PV27,West-Nederland,LD03
4,4,WP1600,WP1600,Haarlemmermeer,GM0394,Noord-Holland,PV27,West-Nederland,LD03


In [225]:
# got this function from stack, allows us to get wikidata

def get_city_wikidata(city, country):
    query = """
    SELECT ?city ?cityLabel ?country ?countryLabel ?population ?coordinates
    WHERE
    {
      ?city rdfs:label '%s'@en.
      ?city wdt:P1082 ?population.
      ?city wdt:P17 ?country.
      ?city rdfs:label ?cityLabel.
      ?country rdfs:label ?countryLabel.
      ?city wdt:P625  ?coordinates.
      FILTER(LANG(?cityLabel) = "en").
      FILTER(LANG(?countryLabel) = "en").
      FILTER(CONTAINS(?countryLabel, "%s")).
    }
    """ % (city, country)

    res = qwikidata.sparql.return_sparql_query_results(query)
    out = res['results']['bindings'][0]
    return out



In [232]:
# input for funcution is all the places in the column 'Woonplaats' of the cities df

def get_city_info():
    
    cities = pd.read_csv("WoonplaatsenNL.csv", sep=";")
    
    #lijst = list(cities['Woonplaats'])
    
    pop = []
    long = []
    lat = []
    for place in cities['Woonplaats']:
        # the place has a whitespace after the name, remove that first
        place = place.rstrip()
        
        try:
            info = get_city_wikidata(place, 'Netherlands')
            popu_info = info['population']
            popu = int(popu_info.get('value'))
            pop.append(popu)
            longlat_info = info.get('coordinates')
            longlat = longlat_info.get('value')
            longlat = longlat.replace('Point(', '')
            longlat = longlat.replace(')','')
            cor = longlat.split()
            long.append(float(cor[0]))
            lat.append(float(cor[1]))

        except:
            pop.append('No info')
            
            try:
                info = get_city_wikidata(place, 'Netherlands')
                longlat_info = info.get('coordinates')
                longlat = longlat_info.get('value')
                longlat = longlat.replace('Point(', '')
                longlat = longlat.replace(')','')
                cor = longlat.split()
                long.append(float(cor[0]))
                lat.append(float(cor[1]))
            except:
                long.append('No info')
                lat.append('No info')
    
    #print(pop)
    #print(lat)
    #print(long)
    # how the f do I add the listst to the columns of the cities df that's outside this function?
    # or do I need to create the df in this function?
    
    cities["Latitude"] = lat
    cities["Longtitude"] = long
    cities["Population"] = pop
    
    cities.to_csv('City_info.csv')
    
    return cities

        
    

        
    

In [233]:
get_city_info()


,ID,Woonplaatsen,Woonplaatscode_1,Woonplaats,Code_3,Provincie,Code_5,Deel,Code_7,Latitude,Longtitude,Population
0,0,WP1925,WP1925,Almelo,GM0141,Overijssel,PV23,Oost-Nederland,LD02,52.3667,6.66667,72459
1,1,WP2774,WP2774,Veere,GM0717,Zeeland,PV29,West-Nederland,LD03,51.5469,3.5403,21865
2,2,WP2145,WP2145,Coevorden,GM0109,Drenthe,PV22,Noord-Nederland,LD01,52.6667,6.75,35483
3,3,WP2571,WP2571,Aalsmeer,GM0358,Noord-Holland,PV27,West-Nederland,LD03,52.2639,4.7625,31728
4,4,WP1600,WP1600,Haarlemmermeer,GM0394,Noord-Holland,PV27,West-Nederland,LD03,52.3008,4.6653,154235
...,...,...,...,...,...,...,...,...,...,...,...,...
2495,2495,WP1367,WP1367,Waadhoeke,GM1949,Friesland,PV21,Noord-Nederland,LD01,No info,No info,No info
2496,2496,WP3029,WP3029,Midden-Drenthe,GM1731,Drenthe,PV22,Noord-Nederland,LD01,No info,No info,No info
2497,2497,WP2399,WP2399,Zwijndrecht,GM0642,Zuid-Holland,PV28,West-Nederland,LD03,51.824,4.61258,44454
2498,2498,WP2169,WP2169,Coevorden,GM0109,Drenthe,PV22,Noord-Nederland,LD01,No info,No info,No info


In [235]:
get_city_wikidata('Coevorden', 'Netherlands')

{'city': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q60453'},
 'population': {'datatype': 'http://www.w3.org/2001/XMLSchema#decimal',
  'type': 'literal',
  'value': '35483'},
 'coordinates': {'datatype': 'http://www.opengis.net/ont/geosparql#wktLiteral',
  'type': 'literal',
  'value': 'Point(6.75 52.666666666)'},
 'country': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q55'},
 'cityLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Coevorden'},
 'countryLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Netherlands'}}

In [203]:
# using cities['Woonplaats'] doesn't work as input so I created a list. Maybe I'll add this to the function

lijst = list(cities['Woonplaats'])
print(lijst[11].strip())

Alphen aan den Rijn


In [211]:
# add the lists of function to the cities df

cities["Latitude"] = lat
cities["Longtitude"] = long
cities["Population"] = pop
cities.head(10)

ValueError: Length of values does not match length of index

In [ ]:
# Backup for long lat if not all is gathered
# alsof unction from stack btw
geolocator = Nominatim(user_agent="http")
country ="NL"

lat = []
long = []

for place in cities['Woonplaats']:
    city = place
    loc = geolocator.geocode(city+','+ country)
    lat.append(loc.latitude)
    long.append(loc.longitude)
    print(lat,long)
#print(city, "latitude is :-" ,loc.latitude,"\nlongtitude is:-" ,loc.longitude)

#cities.loc[city] = 

cities["Latitude"] = lat
cities["Longtitude"] = long